In [ ]:
# default_exp pds.indexes

# PDS Indexes

> Support tools to work with PDS index files. 

In [ ]:
# hide
from nbdev.showdoc import show_doc

In [ ]:
# export
import logging
from datetime import datetime
from pathlib import Path
from urllib.parse import urlsplit, urlunsplit
from urllib.request import URLError

import tomlkit as toml
from dateutil import parser

import pandas as pd
from planetarypy import utils
from planetarypy.config import config
from planetarypy.pds.ctx_index import CTXIndex
from planetarypy.pds.lroc_index import LROCIndex
from planetarypy.pds.utils import IndexLabel

logger = logging.getLogger(__name__)

storage_root = Path(config.storage_root)

In [ ]:
# export
dynamic_urls = {"mro.ctx": CTXIndex, "lro.lroc": LROCIndex}

In [ ]:
# hide
# TODO: HiRISE EDR CumIndex needs to run fixer from pds.utils

In [ ]:
# hide
storage_root

Path('/home/maye/big_drive/planetary_data')

In [ ]:
# export
class Index:
    """Index manager class.

    This class manages one index, identified by a dotted key, e.g. cassini.iss.ring_summary
    """

    def __init__(
        self,
        key: str,  # Nested (dotted) key, e.g. cassini.iss.ring_summary
        url: str = None,  # URL to index. If not given, will be read from config object.
    ):
        self.key = self.parse_key(key)
        self.set_url(url)
        try:
            self.timestamp = parser.parse(config.get_value(self.key)["timestamp"])
        except toml.exceptions.NonExistentKey:
            if self.local_label_path.exists():
                self.timestamp = datetime.fromtimestamp(
                    self.local_label_path.stat().st_mtime
                )
                self.update_timestamp()
            else:
                self.timestamp = None
        self.new_timestamp = None  # filled by needs_download()

    def parse_key(
        self,
        key: str,  # dotted key
    ):
        """Take care of different ways how the key could be structured.

        This involves adding the sub-key `indexes` for the config file structure,
        which is something the user of this class should not need to know.
        """
        tmp = key if key.startswith("missions") else "missions." + key
        subs = tmp.split(".")
        if subs[3] != "indexes":
            subs.insert(3, "indexes")
        return ".".join(subs)

    def set_url(self, url):  # URL to index.
        "Set URL from having it dynamically determined (for non-static index URLs)."
        self.url = config.get_value(self.key)["url"] if url is None else url
        if not self.url:  # empty ''
            self.url = dynamic_urls[self.instrument_key]().latest_index_label_url

    @property
    def isotimestamp(self):
        return self.timestamp.isoformat()

    @property
    def needs_download(self):
        """Determine if the index needs to be downloaded.

        Download shall happen when
        (1) no local timestamp was stored or
        (2) when the remote timestamp is newer.

        Parameters
        ----------
        index : indices.Index (namedtuple)
            Index holding the timestamp attribute read from the config file

        Returns
        -------
        bool
            Boolean indicating if download shall happen.
        """
        try:
            self.remote_timestamp = utils.get_remote_timestamp(self.url)
        except URLError:
            return None
        if self.timestamp:
            if self.remote_timestamp > self.timestamp:
                return True
            else:
                return False
        else:
            # also return True when the timestamp is not valid
            return True

    @property
    def key_tokens(self):
        return self.key.split(".")

    @property
    def mission(self):
        return self.key_tokens[1]

    @property
    def mission_key(self):
        return ".".join(self.key_tokens[1:2])

    @property
    def instrument(self):
        return self.key_tokens[2]

    @property
    def instrument_key(self):
        return ".".join(self.key_tokens[1:3])

    @property
    def index_name(self):
        "str: Examples: EDR, RDR, moon_summary"
        return self.key_tokens[3]

    @property
    def label_filename(self):
        return Path(self.url.split("/")[-1])

    @property
    def isupper(self):
        return self.label_filename.suffix.isupper()

    @property
    def table_filename(self):
        new_suffix = ".TAB" if self.isupper else ".tab"
        return self.label_filename.with_suffix(new_suffix)

    @property
    def label_path(self):
        return Path(urlsplit(self.url).path)

    @property
    def table_path(self):
        return self.label_path.with_name(self.table_filename.name)

    @property
    def table_url(self):
        tokens = urlsplit(self.url)
        return urlunsplit(
            tokens._replace(
                path=str(self.label_path.with_name(self.table_filename.name))
            )
        )

    @property
    def local_dir(self):
        p = storage_root / str(self.key).replace(".", "/")
        p.mkdir(parents=True, exist_ok=True)
        return p

    @property
    def local_table_path(self):
        return self.local_dir / self.table_filename

    @property
    def local_label_path(self):
        return self.local_dir / self.label_filename

    @property
    def local_hdf_path(self):
        return self.local_table_path.with_suffix(".hdf")

    @property
    def local_parq_path(self):
        return self.local_table_path.with_suffix(".parq")

    @property
    def df(self):
        return pd.read_hdf(self.local_hdf_path)
    
    @property
    def parquet(self):
        return pd.read_parquet(self.local_parq_path)

    def download(
        self,
        convert_to_hdf: bool = False,  # Switch to enable conversion to HDF
        convert_to_parquet: bool = True,  # Switch to enable conversion to parquet
        force_update: bool = False,  # Switch to enable a fresh download and conversion
    ):
        """Wrapping URLs for downloading PDS indices and their label files."""
        # check timestamp
        ret = self.needs_download
        if ret is None:
            print("Could not check for any index updates, maybe server is offline?")
            return
        if not ret and not force_update:
            print("Stored index is up-to-date.")
            return
        label_url = self.url
        logger.info("Downloading %s." % label_url)
        utils.url_retrieve(label_url, self.local_label_path)
        logger.info("Downloading %s.", self.table_url)
        utils.url_retrieve(self.table_url, self.local_table_path)
        print(f"Downloaded {self.local_label_path} and {self.local_table_path}")
        self.timestamp = self.remote_timestamp
        self.update_timestamp()

        if convert_to_hdf is True:
            try:
                self.convert_to_hdf()
                self.convert_to_parqet()
                print(f"Converted to pandas HDF:\n{self.local_hdf_path}")
            except:  # any conversion error simpy leads to HDF marked as missing
                self.set_hdf_available(False)
            else:
                self.set_hdf_available(True)

    def set_hdf_available(self, status):
        config.set_value(f"{self.key}.hdf_available", status)

    def update_timestamp(self):
        # Note: the config object writes itself out after setting any value
        config.set_value(f"{self.key}.timestamp", self.isotimestamp)

    def read_index_data(self):
        label = IndexLabel(self.local_label_path)
        df = label.read_index_data()
        return df

    def convert_to_hdf(self):
        df = self.read_index_data()
        df.to_hdf(self.local_hdf_path, "df")

    def convert_to_parquet(self):
        df = self.read_index_data()
        df = df.convert_dtypes()
        df.to_parquet(self.local_parq_path)

    def __str__(self):
        s = f"Key: {self.key}\n"
        s += f"URL: {self.url}\n"
        s += f"Timestamp: {self.timestamp}\n"
        return s

    def __repr__(self):
        return self.__str__()

In [ ]:
show_doc(Index.parse_key)

<h4 id="Index.parse_key" class="doc_header"><code>Index.parse_key</code><a href="__main__.py#L27" class="source_link" style="float:right">[source]</a></h4>

> <code>Index.parse_key</code>(**`key`**:`str`)

Take care of different ways how the key could be structured.

This involves adding the sub-key `indexes` for the config file structure,
which is something the user of this class should not need to know.

**Parameters:**


 - **`key`** : *`<class 'str'>`*	<p>dotted key</p>



In [ ]:
show_doc(Index.download)

<h4 id="Index.download" class="doc_header"><code>Index.download</code><a href="__main__.py#L161" class="source_link" style="float:right">[source]</a></h4>

> <code>Index.download</code>(**`convert_to_hdf`**:`bool`=*`True`*, **`convert_to_parquet`**:`bool`=*`True`*, **`force_update`**:`bool`=*`False`*)

Wrapping URLs for downloading PDS indices and their label files.

**Parameters:**


 - **`convert_to_hdf`** : *`<class 'bool'>`*, *optional*	<p>Switch to enable conversion to HDF</p>


 - **`convert_to_parquet`** : *`<class 'bool'>`*, *optional*	<p>Switch to enable conversion to parquet</p>


 - **`force_update`** : *`<class 'bool'>`*, *optional*	<p>Switch to enable a fresh download and conversion</p>



In [ ]:
key = "missions.cassini.iss.indexes.moon_summary"

In [ ]:
index = Index(key)

In [ ]:
index

Key: missions.cassini.iss.indexes.moon_summary
URL: https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_moon_summary.lbl
Timestamp: 2021-07-23 22:11:38.904571

The `key` can be provided
* with or without the leading 'mssions'
* with or without the internal `indexes` level of the config file

In [ ]:
Index("cassini.iss.indexes.moon_summary")

Key: missions.cassini.iss.indexes.moon_summary
URL: https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_moon_summary.lbl
Timestamp: 2021-07-23 22:11:38.904571

In [ ]:
Index("cassini.iss.moon_summary")

Key: missions.cassini.iss.indexes.moon_summary
URL: https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_moon_summary.lbl
Timestamp: 2021-07-23 22:11:38.904571

In [ ]:
index.isotimestamp

'2021-07-23T22:11:38.904571'

In [ ]:
index.mission_key

'cassini'

In [ ]:
index.local_dir

Path('/home/maye/big_drive/planetary_data/missions/cassini/iss/indexes/moon_summary')

In [ ]:
index.needs_download

False

In [ ]:
index.download()

Stored index is up-to-date.


In [ ]:
index.convert_to_hdf()

Done.


In [ ]:
index.convert_to_parquet()

Done.


In [ ]:
index.df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 385719 entries, 0 to 385718
Data columns (total 35 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   VOLUME_ID                             385719 non-null  object 
 1   FILE_SPECIFICATION_NAME               385719 non-null  object 
 2   OPUS_ID                               385719 non-null  object 
 3   TARGET_NAME                           385719 non-null  object 
 4   MINIMUM_PLANETOCENTRIC_LATITUDE       385719 non-null  float64
 5   MAXIMUM_PLANETOCENTRIC_LATITUDE       385719 non-null  float64
 6   MINIMUM_PLANETOGRAPHIC_LATITUDE       385719 non-null  float64
 7   MAXIMUM_PLANETOGRAPHIC_LATITUDE       385719 non-null  float64
 8   MINIMUM_IAU_LONGITUDE                 385719 non-null  float64
 9   MAXIMUM_IAU_LONGITUDE                 385719 non-null  float64
 10  MINIMUM_LOCAL_HOUR_ANGLE              385719 non-null  float64
 11  

In [ ]:
index.key_tokens[:2]

['missions', 'cassini']

In [ ]:
index.instrument_key

'cassini.iss'

In [ ]:
config.get_value(index.instrument_key)

{'indexes': {'index': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_index.lbl'}, 'inventory': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_inventory.lbl'}, 'moon_summary': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_moon_summary.lbl', 'timestamp': '2021-07-23T22:11:38.904571'}, 'ring_summary': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_ring_summary.lbl'}, 'saturn_summary': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_saturn_summary.lbl'}}}

In [ ]:
index = Index("mro.ctx.edr")

In [ ]:
index

Key: missions.mro.ctx.indexes.edr
URL: https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_4044/index/cumindex.lbl
Timestamp: 2021-07-21 19:55:32.014076

In [ ]:
index.download()

Stored index is up-to-date.
